In [1]:
from Config import Config
from torch.utils.data import DataLoader
import torch
import wandb
import gc
from lightning.pytorch.loggers import WandbLogger
import os
from lightning import Trainer, seed_everything
from dataloader import CustomTextDataset
import argparse
import pandas as pd
from Seq2SeqModel import Seq2SeqModel
import seaborn as sns
import json
from utils import decoder_function
import torch.nn.functional as F

In [2]:
## Dataloader
TEST_BATCH_SIZE = 2
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
DATASET_PATH = os.path.join("..","dataset", "dakshina_dataset_v1.0", "ta", "lexicons")
test_df = pd.read_csv(os.path.join(DATASET_PATH, "ta.translit.sampled.test.idx.csv"))

with open(os.path.join(DATASET_PATH,"tamil_token_index.json"),'r', encoding="utf-8") as f:
	tamil_idx = json.load(f)
tamil_idx_to_char = {j:i for i,j in tamil_idx.items()}

with open(os.path.join(DATASET_PATH,"english_token_index.json"),'r', encoding="utf-8") as f:
	english_idx = json.load(f)
english_idx_to_char = {j:i for i,j in english_idx.items()}

english_idx_to_char[26] = "-"
tamil_idx_to_char[48] = "-"


config = Config()

test_dataset = CustomTextDataset(
        dataset_df=test_df,
        X_max_length=config.X_max_length,
        Y_max_length=config.Y_max_length,
        X_vocab_size=config.X_vocab_size,
        Y_vocab_size=config.Y_vocab_size,
        X_padding_idx=config.X_padding_idx,
        Y_padding_idx=config.Y_padding_idx,
    )

lit_model = Seq2SeqModel(config=config)

In [3]:
X, Y_decoder_ip, Y_decoder_op, X_len, Y_decoder_ip_len, Y_decoder_op_len = test_dataset.__getitem__(0)

X_native = decoder_function(character_idx_seq=','.join([str(i) for i in X.detach().numpy()]), idx_to_char_dict=english_idx_to_char)
Y_native = decoder_function(character_idx_seq=','.join([str(i) for i in Y_decoder_op.detach().numpy()]), idx_to_char_dict=tamil_idx_to_char)



X = torch.unsqueeze(X, 0).to(device)
X_len = torch.unsqueeze(X_len, 0).to(device)
Y_decoder_ip = torch.unsqueeze(Y_decoder_ip, 0).to(device)

print(X.shape)
print(X_native)
print(Y_native)

torch.Size([1, 30])
faarm-------------------------
ஃபார்ம்
-------------------


In [4]:
(logits, attn_weight_list) = lit_model(X, X_len, Y_decoder_ip)
logits = logits.view(-1, logits.size(-1))
prob = F.softmax(logits, dim=1)
preds = torch.argmax(prob, dim=1)
Y_pred_native = decoder_function(character_idx_seq=','.join([str(i) for i in preds.detach().numpy()]), idx_to_char_dict=tamil_idx_to_char)
attn_weight_list = attn_weight_list.squeeze(0)# reshaped to [Y_max_len,X_max_len] #.permute(1,0) 

In [5]:
#attn_weight_list[:X_len.cpu().detach().numpy(),:Y_decoder_ip_len].cpu().detach().numpy()
import matplotlib.pyplot as plt
from matplotlib import font_manager

#plt.rc('font', family='Vijaya')  # Or another Tamil-supporting font installed on your system
# Path to your downloaded NotoSansTamil-Regular.ttf
font_path = r'C:\Windows\Fonts\NotoSansTamil-Regular.ttf'
#C:\Users\yuvar\AppData\Local\Microsoft\Windows\Fonts
# Add font to Matplotlib
font_manager.fontManager.addfont(font_path)

# Set font family
plt.rcParams['font.family'] = 'Noto Sans Tamil'
data = attn_weight_list[:Y_decoder_ip_len,:X_len].detach().numpy()
sns.heatmap(data, xticklabels=X_native[:X_len], yticklabels=Y_pred_native[:Y_decoder_ip_len])


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Windows\\Fonts\\NotoSansTamil-Regular.ttf'